In [74]:
from collections import deque
from dataclasses import dataclass

import numpy as np
import mesa


MAX_GROUPS = 10
GATE = "G"
OBSTACLE = "#"
GROUP = [str(i) for i in range(MAX_GROUPS)]
EMPTY = " "

MAP_SYMBOLS = {str(i): i for i in range(MAX_GROUPS)}
MAP_SYMBOLS[GATE] = 100
MAP_SYMBOLS[OBSTACLE] = -1
MAP_SYMBOLS[EMPTY] = 0

MAP_VALUES = {v: k for k, v in MAP_SYMBOLS.items()}


In [8]:
def agent_portrayal(agent):
    portrayal = {
        "Shape": "circle",
        "Color": "red",
        "Filled": "true",
        "Layer": 0,
        "r": 0.5,
    }
    return portrayal


In [6]:
class LeaderAgent(mesa.Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name

    def step(self):
        print("{} activated".format(self.name))

class Cell(mesa.Agent, x, y):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name


class RoomModel(mesa.Model):
    def __init__(self, n_agents, width, height, gate):
        super().__init__()
        self.schedule = mesa.timeRandomActivation(self)
        self.grid = mesa.space.MultiGrid(10, 10, torus=False)
        self.gate = gate
        for x in range(width):
            for y in range(height):
                coords = (x, y)
                cell = Cell(coords)
                self.grid.place_agent(cell, coords)
                
        for i in range(n_agents):
            a = LeaderAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)
    def static_potential(coords):
        
            
    def step(self):
        self.schedule.step()


In [77]:
def create_map(width, height, gate=None):
    if width < 3 or height < 3:
        raise ValueError("Map cannot have dimensions lower than 3 due to walls")
    dimensions = (height, width)
    grid = np.zeros(shape=dimensions)
    grid[:,0] = -1
    grid[:,-1] = -1
    grid[0, :] = -1
    grid[-1, :] = -1
    if gate:
        gate = (gate[1], gate[0])
        grid[gate] = 100
    return grid

In [80]:
def load_map(filename):
    grid = None
    with open(filename) as f:
        dimensions = list(map(int, f.readline().split()))
        width, height = dimensions
        dimensions = height, width
        grid = np.zeros(shape=dimensions)
        for row in range(height):
            line = f.readline().strip()
            for column, c in enumerate(line):
                if c not in MAP_SYMBOLS:
                    raise ValueError("Unknown character when loading map from file")
                np_coords = (row, column)
                grid[np_coords] = MAP_SYMBOLS[c]
    return grid

In [81]:
grid = create_map(10, 10, gate=(3, 5))
print(grid)
save_map(grid, "map1.txt")
print(load_map("map1.txt"))

[[ -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0. 100.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]]
[[ -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0. 100.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.  -1.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0

In [43]:
def static(grid, normalize=True):
    @dataclass
    class Node():
        coords: (int, int)
        obstacle: bool
        price: int = float("inf")
        
        def enter(self, parent):
            self.price = parent.price + self.distance(parent)
        
        def distance(self, other):
            sigma = np.power(self.coords[0] - other.coords[0], 2)
            sigma += np.power(self.coords[1] - other.coords[1], 2)
            return np.sqrt(sigma)
            
        def neighbours(self, width, height):
            valid_coords = []
            for x in [-1, 0, 1]:
                for y in [-1, 0, 1]:
                    if 0 <= self.coords[0] + x < width and 0 <= self.coords[1] + y < height:
                        if x == 0 and y == 0:
                            continue
                        valid_coords.append((self.coords[0]+x, self.coords[1]+y))
            return valid_coords          
        
    q = deque()
    grid_nodes = []
    gate = None
    height, width = grid.shape
    print(grid.shape)
    for y in range(height):
        grid_nodes.append([])
        for x in range(width):
            coords = (x, y)
            np_coords = (y, x)
            node = Node(coords, grid[np_coords] < 0)
            if grid[np_coords] == 100:
                gate = Node(coords, False)
                node = gate
            grid_nodes[y].append(node)
            
    if not gate:
        raise ValueError("Gate is not present in the map. Can't compute static field.")
    
    gate.price = 0
    q.append(gate)
    while q:
        current_node = q.pop()
        while current_node.obstacle:
            current_node = q.pop()
        for coords in current_node.neighbours(width, height):
            x, y = coords
            np_coords = (y, x)
            other_node = grid_nodes[y][x]
            if not other_node.obstacle:
                distance = current_node.distance(other_node)
                if current_node.price + distance < other_node.price:
                    other_node.enter(current_node)
                    q.append(other_node)
    static_field = np.zeros(grid.shape)
    for x in range(width):
        for y in range(height):
            np_coords = (y, x)
            static_field[np_coords] = grid_nodes[y][x].price
    if normalize:
        return  static_field / np.nanmax(static_field[static_field != np.inf])
    return static_field

In [ ]:
grid = mesa.visualization.CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = mesa.visualization.ModularServer(
    MoneyModel, [grid], "Money Model", {"N": 100, "width": 10, "height": 10}
)
server.port = 8521  # The default
server.launch()